### Проверяем качество работы морфологического и синтаксического анализаторов SpaCy на корпусах из Universal Dependencies, размеченных в формате CoNLL-U
Подробнее: https://universaldependencies.org/

In [3]:
import conllu
import spacy
import pandas as pd
import re

In [4]:
def get_corpus(file):
    f = open(file, 'r', encoding='UTF-8')
    return [tokenlist for tokenlist in conllu.parse_incr(f)]

In [5]:
syntagrus = get_corpus('ru_syntagrus-full.conllu')
taiga = get_corpus('ru_taiga-full.conllu')
pud = get_corpus('ru_pud-ud-test.conllu')
gsd = get_corpus('ru_gsd-full.conllu')

* получили списки с предложениями
* предложние - объект класса conllu.models.TokenList; хранит токены
* токен - объект класса conllu.models.Token; хранит грамматическую и синтаксическую информацию

In [6]:
nlp = spacy.load('ru_core_news_lg')

Поскольку нет задачи проверить качество токенизации, возьмем только те предложения, токены которых совпадают в корпусе и после токенизации SpaCy. Остальные невозможно будет сравнить.

In [7]:
def filter_tokens(corpus):
    
    
    output_list = []
    
    for sent in corpus:
        actual_tokens = [re.findall('\w+', w['form']) for w in sent]
        spacy_tokens = [re.findall('\w+', w.text) for w in nlp(sent.metadata['text'])]
        
        if actual_tokens == spacy_tokens:
            output_list.append([nlp(sent.metadata['text']), sent])
            
    return output_list

После этого сравниваем леммы, части речи, грамматические характеристики (последние сравниваем при условии совпадения частей речи).

In [8]:
def compare_morph(corpus):
    
    
    sim_lemmas = 0
    all_lemmas = 0
    sim_pos = 0
    all_pos = 0
    sim_tags = 0
    all_tags = 0
    
    for pair in corpus:
        for i in range(len(pair[0])):
            
            if pair[0][i].lemma_.lower() == pair[1][i]['lemma'].lower():
                sim_lemmas += 1
            
            if pair[0][i].pos_ == pair[1][i]['upos']:
                sim_pos += 1
                   
                if pair[0][i].morph.to_dict() != {} and pair[1][i]['feats'] != None:
                      
                    if 'StyleVariant' in pair[0][i].morph.to_dict() and 'Variant' in pair[1][i]['feats']:
                        
                        if pair[0][i].morph.to_dict()['StyleVariant'] == pair[1][i]['feats']['Variant']:
                            sim_tags+=1
                                
                    if 'Person' in pair[0][i].morph.to_dict() and 'Person' in pair[1][i]['feats']:
                        temp_dict = pair[0][i].morph.to_dict()
                        
                        if temp_dict['Person'] == 'First' and pair[1][i]['feats']['Person'] == '1':
                            temp_dict['Person'] = '1'
                            
                        elif temp_dict['Person'] == 'Second' and pair[1][i]['feats']['Person'] == '2':
                            temp_dict['Person'] = '2'
                        
                        elif temp_dict['Person'] == 'Third' and pair[1][i]['feats']['Person'] == '3':
                            temp_dict['Person'] = '3'
                        
                        if temp_dict == pair[1][i]['feats']:
                            sim_tags+=1           
                        
                    if pair[0][i].morph.to_dict() == pair[1][i]['feats']:
                        sim_tags += 1
                
                if pair[0][i].morph.to_dict() == {} and pair[1][i]['feats'] == None:
                    sim_tags += 1
            
            all_lemmas += 1
            all_pos += 1
            all_tags += 1  
    
    acc_lemmas = sim_lemmas / all_lemmas
    acc_pos = sim_pos / all_pos
    acc_tags = sim_tags / all_tags
    
    return acc_lemmas, acc_pos, acc_tags

Сравниваем главные слова (heads) каждого зависимого слова и находим их отношения (deprels). Метрика UAS - доля совпавших heads, LAS - доля совпавших deprels при условии совпадения heads.

In [9]:
def compare_syntax(corpus):
    
    
    sim_heads = 0
    sim_deprels = 0
    heads = 0
    
    for pair in corpus:
        for i in range(len(pair[0])):
               
            if pair[1][i]['head'] == pair[0][i].head.i+1 \
            or (pair[1][i]['head'] == 0 and pair[1][i]['form'] == pair[0][i].head.text):
                sim_heads += 1
                    
                if pair[1][i]['deprel'] == pair[0][i].dep_ \
                or (pair[1][i]['deprel'] == 'root' and pair[0][i].dep_ == 'ROOT'):
                    sim_deprels += 1
            heads+=1
                
    UAS = sim_heads / heads
    LAS = sim_deprels / heads
    return UAS, LAS            

In [10]:
def calculate(corpus):
    filtered = filter_tokens(corpus)
    return compare_morph(filtered), compare_syntax(filtered)

In [11]:
taiga_res = calculate(taiga)
pud_res = calculate(pud)
gsd_res = calculate(gsd)
syntagrus_res = calculate(syntagrus)

Accuracy, морфология

In [16]:
df_morph = pd.DataFrame()
df_morph['gsd'] = gsd_res[0]
df_morph['taiga'] = taiga_res[0]
df_morph['pud'] = pud_res[0]
df_morph['syntagrus'] = syntagrus_res[0]
df_morph.rename({0: 'acc_lemma', 1: 'acc_pos', 2: 'acc_features'})

,gsd,taiga,pud,syntagrus
acc_lemma,0.929845,0.916053,0.913988,0.923443
acc_pos,0.956143,0.933273,0.971534,0.959689
acc_features,0.839950,0.766308,0.917600,0.854662


Accuracy, синтаксис

In [17]:
df_syntax = pd.DataFrame()
df_syntax['gsd'] = gsd_res[1]
df_syntax['taiga'] = taiga_res[1]
df_syntax['pud'] = pud_res[1]
df_syntax['syntagrus'] = syntagrus_res[1]
df_syntax.rename({0: 'UAS', 1: 'LAS'})

,gsd,taiga,pud,syntagrus
UAS,0.873937,0.829492,0.930762,0.887058
LAS,0.813382,0.777022,0.887114,0.838188
